In [2]:
import numpy as np
import torch
import torchvision

# about torch

### 0. load model

In [ ]:
DEBUG = True

class vgg():
    
    def __init__(self):
        self.model_path = '../faster_rcnn/data/pretrained_model/vgg16_caffe.pth'

    def _init_modules(self):
        vgg = torchvision.models.vgg16()
        if DEBUG:
            # print(type(vgg.state_dict()))   # <class 'collections.OrderedDict'>
            for i, _ in vgg.state_dict().items():
                print(i)

        load_state_dict = torch.load(model_path)
        if DEBUG:
            print('****************')
            # print(type(state_dict))  # <class 'collections.OrderedDict'>
            for i, _ in load_state_dict.items():
                print(i)
        
        if DEBUG:
            print('****************')
            for para in vgg.parameters():
                # print(type(para))
                print(para.size())
                
vgg16 = vgg()
vgg16._init_modules()

### 1. DataLoader

In [13]:
import torch.utils.data as data
from torch.utils.data.sampler import Sampler

In [25]:
class MyDataset(data.Dataset):
    def __init__(self):
        self.data = torch.eye(7)
        self.data_len = self.data.size(0)
        
    def __getitem__(self, index):
        print(index)
        return self.data[index]
    
    def __len__(self):
        return int(self.data_len)
    
class Samp(Sampler):
    def __init__(self):
        self.smp = np.array([0, 1, 4, 6, 3, 2, 5])
        self.smp_len = len(self.smp)
    
    def __iter__(self):
        return iter(self.smp)
    
    def __len__(self):
        return self.smp_len


mydataset = MyDataset()    
samp = Samp()

dataloader = data.DataLoader(mydataset,
                             batch_size=3,
                             shuffle=False,
                             sampler=samp)

for i, v in enumerate(dataloader):
    print(v)

0
1
4
tensor([[1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.]])
6
3
2
tensor([[0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]])
5
tensor([[0., 0., 0., 0., 0., 1., 0.]])


### 2. Tensor

In [59]:
t1 = torch.randn(3, 3)
print(t1.size(0), type(t1.size(0)))

t2 = torch.tensor(3)
t3 = torch.tensor([[7]])
print(t2.size(), t3.size())
print(t2.item())
print(t3.item())

3 <class 'int'>
torch.Size([]) torch.Size([1, 1])
3
7


### 3. some func api

In [3]:
s1 = torch.randn(3, 3)
s2 = torch.max(s1, 1)
print(s1)
print(s2)

s3 = torch.randn(7, 5, 4)
print(s3)
s4 = torch.max(s3, 2)
print(s4)
s5 = torch.max(s3, 1)
print(s5)

tensor([[ 0.9422,  2.2302, -0.6731],
        [ 0.1782,  1.0912, -0.5894],
        [ 0.5980,  0.5205, -0.3998]])
torch.return_types.max(
values=tensor([2.2302, 1.0912, 0.5980]),
indices=tensor([1, 1, 0]))
tensor([[[-1.2189,  0.3860,  0.6612, -0.5503],
         [ 0.0235,  1.4213,  0.8730,  0.1270],
         [ 1.7497, -0.7082, -1.4129,  0.4653],
         [-0.8298,  0.3792, -1.8405,  0.1535],
         [ 0.2550, -1.4524, -0.5629,  1.0850]],

        [[-1.8875,  0.9664,  0.8628, -0.9391],
         [-1.7569,  1.2431,  0.1463, -0.4209],
         [-1.3757, -0.4443, -1.6345,  1.6738],
         [-0.1588, -1.8369,  0.5406,  0.8178],
         [-0.9371,  0.5605, -0.4585,  0.4215]],

        [[-1.6968, -0.5511,  0.7867,  0.4336],
         [-0.5813, -0.3624, -0.3887,  0.9923],
         [ 0.1936,  0.7338, -0.9608,  0.6367],
         [-0.1973,  0.3087, -1.5334, -1.3575],
         [-0.8923, -0.0491,  0.4147,  0.3574]],

        [[ 0.3110, -0.2247,  0.2613,  1.6110],
         [-1.6648,  1.1230,  0.4536, -

In [15]:
m1 = torch.tensor([[1, 2]])
m2 = torch.tensor([[2, 4]])
m3 = torch.pow(m1 - m2, 2)
print(m3)
m4 = torch.sum(m3, -1, keepdim=True).float()
print(m4)
print(torch.sqrt(m4))
print(torch.mean(m4, -1, keepdim=True), torch.mean(m4))
m5 = torch.randn(2, 3)
print(torch.cat([m5, m5]))

tensor([[1, 4]])
tensor([[5.]])
tensor([[2.2361]])
tensor([[5.]]) tensor(5.)
tensor([[-0.0459, -2.5168,  0.4648],
        [ 1.2245, -0.1432,  0.9072],
        [-0.0459, -2.5168,  0.4648],
        [ 1.2245, -0.1432,  0.9072]])


### 4. about dynamic graph

In [16]:
import torch

In [24]:
x = torch.ones(2, 3, requires_grad=True)

y = x + 2
z = y * y * 3
out = z.mean()

# after backward, graph will be destroyed
out.backward()
# out.backward()
print(x.grad)

tensor([[3., 3., 3.],
        [3., 3., 3.]])


In [32]:
class Net1:
    def __init__(self, x):
        # x.size() -> (3, 2)
        self.w = torch.ones(2, 3, requires_grad=True)
        self.x = x
    def forward(self):
        out = torch.mm(self.x, self.w)  #(3,3)
        out = out * out
        return out

x = torch.ones(3, 2)
net = Net1(x)
for i in range(5):
    out = net.forward()
    result = out.mean()
    result.backward()
    print(net.w.grad)
    net.w.grad.zero_()

tensor([[1.3333, 1.3333, 1.3333],
        [1.3333, 1.3333, 1.3333]])
tensor([[1.3333, 1.3333, 1.3333],
        [1.3333, 1.3333, 1.3333]])
tensor([[1.3333, 1.3333, 1.3333],
        [1.3333, 1.3333, 1.3333]])
tensor([[1.3333, 1.3333, 1.3333],
        [1.3333, 1.3333, 1.3333]])
tensor([[1.3333, 1.3333, 1.3333],
        [1.3333, 1.3333, 1.3333]])
